In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
# Adapter Layer

def read_csv_to_df(filename, bucket):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df

def write_df_to_s3(trg_bucket, key, df_all, s3):
    out_buffer = BytesIO()
    df_all.to_parquet(out_buffer, index=False)
    bucket_target = s3.Bucket(trg_bucket)
    bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)
    return bucket_target

def return_objects(src_bucket, src_format,arg_date, bucket):
    arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]
    ##objKey= obj.key
    return objects

In [3]:
# Application Layer

def extract(objects, bucket, columns, key):
    df_all = pd.concat([read_csv_to_df(obj.key, bucket) for obj in objects], ignore_index=True)
    df_all = df_all.loc[:, columns]
    df_all.dropna(inplace=True)
    return df_all

def transform_report(df_all, arg_date, columns):
    df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
    df_all.drop(columns=['prev_closing_price'], inplace=True)
    df_all = df_all.round(decimals=2)
    df_all = df_all[df_all.Date >= arg_date]
    return df_all

def load(bucket_target, key, s3):
    
    objKey=[]
    for obj in bucket_target.objects.all():
        objKey.append(obj.key)
    prq_obj = bucket_target.Object(key=objKey[-1]).get().get('Body').read()
    data = BytesIO(prq_obj)
    return data

def etl_report(bucket_target, key, s3):
    df_report = pd.read_parquet(load(bucket_target, key, s3))
    return df_report
    

In [4]:
# main function entrypoint

def main():
    # Parameters/Configurations    
    
    arg_date = input("Introduce una fecha (ej: 2022-03-14): ")
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-bucket-idn'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
    
     # Init    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(src_bucket)
    objects = return_objects(src_bucket, src_format,arg_date, bucket)
    df_all = extract(objects, bucket, columns, key)
    df_all =  transform_report(df_all, arg_date, columns)
    bucket_target = write_df_to_s3(trg_bucket, key, df_all, s3)
    
    # run application
    print(etl_report(bucket_target, key, s3))
    

In [5]:
# run

main()

Introduce una fecha (ej: 2022-03-14): 2022-03-14
              ISIN        Date  opening_price_eur  closing_price_eur  \
0     AT000000STR1  2022-03-14              35.45              36.25   
1     AT00000FACC2  2022-03-14               7.83               7.84   
2     AT0000606306  2022-03-14              13.66              13.27   
3     AT0000609607  2022-03-14              11.36              11.30   
4     AT0000644505  2022-03-14              91.40              92.70   
...            ...         ...                ...                ...   
3213  XS2427363036  2022-03-14              22.38              21.78   
3214  XS2427363895  2022-03-14              18.58              18.58   
3215  XS2427474023  2022-03-14              24.01              24.25   
3216  XS2434891219  2022-03-14               3.57               3.54   
3217  XS2437455608  2022-03-14              25.92              26.67   

      minimum_price_eur  maximum_price_eur  daily_traded_volume  \
0                 3